In [12]:
import json, transformers, torch
from torch import bfloat16, cuda
from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4'

In [13]:
def load_qa_data(filename):
    with open(filename, 'r') as f:
        data = [json.loads(line) for line in f]
    formatted_data = []
    for item in data:
        item['question'] = ' '.join(item['question'].split())
        formatted_data.append(f"question: {item['question']} answer: {item['answer']}")
    return formatted_data

def load_txt(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return lines
    
dataset = load_txt('chunked.txt')
for x in dataset:
    print(x)

Databases and database technology have had a major impact on the growing use of computers. It is fair to say that databases play a critical role in almost all areas where computers are used, including business, electronic commerce, social media, engi-neering, medicine, genetics, law, education, and library science. The word database is so commonly used that we must begin by defining what a database is. Our initial definition is quite general. A database  is a collection of related data. 1 By data , we mean known facts that can be recorded and that have implicit meaning.

For example, consider the names, telephone numbers, and addresses of the people you know. Nowadays, this data is typically stored in mobile phones, which have their own simple database software. This data can also be recorded in an indexed address book or stored on a hard drive, using a personal computer and software such as Microsoft Access or Excel. This collection of related data with an implicit meaning is a databa

In [14]:
from peft import LoraConfig, get_peft_model
from datasets import Dataset

"""instatiating the model and quantizing config"""

model_name = 'meta-llama/Llama-2-13b-chat-hf'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype= bfloat16
)

model_path = 'meta-llama/Llama-2-13b-chat-hf'

model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
#lora_config = LoraConfig.from_pretrained('outputs')

#model = get_peft_model(model, lora_config)



tokenizer = transformers.AutoTokenizer.from_pretrained('meta-llama/Llama-2-13b-chat-hf')
tokenizer.pad_token = tokenizer.eos_token

#model.eval()

'''text = "What four main types of actions involve databases? Briefly discuss each."

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=2048)'''

from datasets import Dataset

train_dataset = Dataset.from_dict({
    'text': [item for item in dataset]
})



lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

from transformers import TrainingArguments

output_dir = "./3epoch_results_13b"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 300
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    num_train_epochs = 3
)

from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,

)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/5945 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacty of 47.54 GiB of which 10.75 MiB is free. Including non-PyTorch memory, this process has 47.52 GiB memory in use. Of the allocated memory 47.25 GiB is allocated by PyTorch, and 15.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.train()

model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

Step,Training Loss
10,2.614400
20,2.546000
30,2.424100
40,2.332800
50,2.687600
60,2.268600
70,2.288900
80,2.219700
90,2.298900
100,2.582600


In [ ]:


lora_config = LoraConfig.from_pretrained('outputs')

model = model.to(device)
model = get_peft_model(model, lora_config)
model = model.to(device)

text = "What four main types of actions involve databases? Briefly discuss each."


inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=2048)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

You shouldn't move a model when it is dispatched on multiple devices.


OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacty of 47.54 GiB of which 2.75 MiB is free. Including non-PyTorch memory, this process has 47.46 GiB memory in use. Of the allocated memory 47.15 GiB is allocated by PyTorch, and 15.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
#loss curves
import matplotlib.pyplot as plt
import json

# Load your data
with open('tr')
data = 
parsed_data = json.loads(data)

# Extracting loss and learning rate data
steps = [entry['step'] for entry in parsed_data['log_history']]
losses = [entry['loss'] for entry in parsed_data['log_history']]
learning_rates = [entry['learning_rate'] for entry in parsed_data['log_history']]

# Plotting Loss Curve
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(steps, losses, label='Training Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()

# Plotting Learning Rate
plt.subplot(1, 2, 2)
plt.plot(steps, learning_rates, color='orange', label='Learning Rate')
plt.xlabel('Steps')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Over Time')
plt.legend()

plt.tight_layout()
plt.show()
